<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb tenacity

In [2]:
import os
import random
from pathlib import Path
import openai
import tiktoken
from pprint import pprint
from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [3]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [4]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

wandb: Currently logged in as: darek. Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [5]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [6]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [7]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

"How do I track the performance of different models over time on W&B?"

How do I connect and monitor my machine learning experiments in W&B?

"How do I save my project to a specific directory in W&B?"

"How do I track multiple experiments within a single W&B project?"

"How do I share my W&B project with my team members and give them the ability to collaborate and make changes?"

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [8]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "What are the risk of supplying proprietary data into a third party cloud provider such as weights and 
biases?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [9]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: what is a     
weave query expression that can calculate the median of a column of numbers in a wandb Table can I use the column  
subsample parameter config file for xgboost parameters through sweeps? I'm getting a "Importing a module script    
failed. An application error occurred." error on a computer, but the same link works on my phone. Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [10]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

How can I use W&B to track and analyze the performance of my deep learning model?

How do I visualize the distribution of a specific column in my W&B Table using histograms or density plots?

Sure, here's a support question based on these examples:                                                           

How can I visualize the ROC curve for my model in W&B? What is the format of the input required for the roc_curve  
function in W&B?

How can I use W&B to track and visualize model performance over time when training on multiple datasets with       
different labels?

How can I plot a graph in WandB that shows the trend of validation and training loss during training?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [11]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [12]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[4179, 365, 1206, 2596, 2940, 537, 956, 803, 1644, 2529, 2093]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [13]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [14]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [15]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: 2 = Linear(n_layer_1, n_layer_2) self.layer_3 =
Linear(n_layer_2, n_classes)                                                                                       

    self.loss = CrossEntropyLoss()                                                                                 
    self.lr = lr                                                                                                   
                                                                                                                   
    # save hyper-parameters to self.hparams (auto-logged by W&B)                                                   
    self.save_hyperparameters()                                                                                    
                                                                                                                   
def forward(self, x):                                                                                              
    '''method used for inference input -> output'''                                                                
                                                                                                                   
    # (b, 1, 28, 28) -> (b, 1*28*28)                                                                               
    batch_size, channels, width, height = x.size()                                                                 
    x = x.view(batch_size, -1)                                                                                     
                                                                                                                   
    # let's do 3 x (linear + relu)                                                                                 
    x = F.relu(self.layer_1(x))                                                                                    
    x = F.relu(self.layer_2(x))                                                                                    
    x = self.layer_3(x)                                                                                            
    return x                                                                                                       
                                                                                                                   
def training_step(self, batch, batch_idx):                                                                         
    '''needs to return a loss from a single batch'''                                                               
    _, loss, acc = self._get_preds_loss_accuracy(batch)                                                            
                                                                                                                   
    # Log loss and metric                                                                                          
    self.log('train_loss', loss)                                                                                   
    self.log('train_accuracy', acc)                                                                                
    return loss                                                                                                    
                                                                                                                   
def validation_step(self, batch, batch_idx):                                                                       
    '''used for logging metrics'''                                                                                 
    preds, loss, acc = self._get_preds_loss_accuracy(batch)                                                        
                                                       

Let's generate 3 possible questions:

In [16]:
generate_and_print(system_prompt, generation_prompt, n=3)

What is the purpose of the define_metric function in W&B, and how can it be used to control the display of summary 
metrics such as min, max, mean and best value?

What is the purpose of the _get_preds_loss_accuracy function in this W&B documentation fragment and how is it used 
in the training_step and validation_step methods?

What method should I use in my WandB project to log the min and max values of a specific metric during training?

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [17]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [18]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [19]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [20]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [21]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [22]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [23]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
how should a project setup look like for a team that uses W&B for experiment tracking and artifacts and now want to
use LLMs in the same project. What are best practices? For distributed parallel runs, how can I get the run_id when
I'm on a process other than RANK 0? How should I cite W&B in a manuscript?                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
2 = Linear(n_layer_1, n_layer_2) self.layer_3 = Linear(n_layer_2, n_classes)                                       

    self.loss = CrossEntropyLoss()                                                                                 
    self.lr = lr                                                                                                   
                                                                                                                   
    # save hyper-parameters to self.hparams (auto-logged by W&B)                                                   
    self.save_hyperparameters()                                                                                    
                                                                                                                   
def forward(self, x):                                                                                              
    '''method used for inference input -> output'''                                                                
                                                                                                                   
    # (b, 1, 28, 28) -> (b, 1*28*28)                                                                               
    batch_size, channels, width, height = x.size()                                                                 
    x = x.view(batch_size, -1)                                                                                     
                                                                                                                   
    # let's do 3 x (linear + relu)                                                                                 
    x = F.relu(self.layer_1(x))                                                                                    
    x = F.relu(self.layer_2(x))                                                                                    
    x = self.layer_3(x)                                                                                            
    return x                                                                                                       
                                                                                                                   
def training_step(self, batch, batch_idx):                                                                         
    '''needs to return a loss from a single batch'''                                                               
    _, loss, acc = self._get_preds_loss_accuracy(batch)                                                            
                                                                                                                   
    # Log loss and metric                                                                                          
    self.log('train_loss', loss)                    

In [24]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [25]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [26]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

("A user is using Weights & Biases integration with PyTorch Lightning and wants to log their model's gradients, parameter histograms, and model topology during training.\n",
 'How do I log the gradients, parameter histograms, and topology of my model while training it with PyTorch Lightning and W&B?\n',
 "To log your model's gradients, parameter histograms, and topology while training with PyTorch Lightning and W&B, you can pass your model object to `wandb_logger.watch()` function. Here's how you can do it:\n\n1. Instantiate the `WandbLogger` and pass it to Lightning's `Trainer`:\n\n```python\nwandb_logger = WandbLogger()\ntrainer = Trainer(logger=wandb_logger)\n```\n\n2. Pass your model object to `wandb_logger.watch()`:\n\n```python\nwandb_logger.watch(model)\n```\n\nFor more details, check the PyTorch Lightning [`WandbLogger` documentation](https://pytorch-lightning.readthedocs.io/en/latest/extensions/generated/pytorch_lightning.loggers.WandbLogger.html?highlight=wandblogger).")

In [27]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

In [28]:
wandb.finish()